# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [3]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [7]:
publications = pd.read_csv("D:\Publications.txt", sep='\t',header=0, encoding = "ISO-8859-1")
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2009-07-01,Ground-based Pc5 ULF wave power: Solar wind sp...,Journal of Atmospheric and Solar-Terrestrial P...,NaN,"Pahud, D. M., Rae, I. J., Mann, I. R., Murphy,...",Pahud,https://doi.org/10.1016/j.jastp.2008.12.004
1,2008-12-01,The Upgraded CARISMA Magnetometer Array in the...,Space Science Reviews,NaN,"Mann, I. R., Milling, D. K., Rae, I. J., Ozeke...",Mann,https://doi.org/10.1007/s11214-008-9457-6
2,2008-07-01,Ionospheric localisation and expansion of long...,Geophysical Research Letters,NaN,"Milling, D. K., Rae, I. J., Mann, I. R., Murph...",Milling,https://doi.org/10.1029/2008GL033672
3,2009-12-01,Timing and localization of near-Earth tail and...,Journal of Geophysical Research: Space Physics,NaN,"Gabrielse, C., Angelopoulos, V., Runov, a., Fr...",Gabrielse,https://doi.org/10.1029/2008JA013583
4,2009-05-01,THEMIS observation of a substorm event on 04:3...,Annales Geophysicae,NaN,"Liu, J., Angelopoulos, V., Frey, H., McFadden,...",Liu,https://doi.org/10.5194/angeo-27-1831-2009
5,2009-01-01,Timing and localization of ionospheric signatu...,Journal of Geophysical Research: Space Physics,NaN,"Rae, I. J., Mann, I. R., Murphy, K. R., Millin...",Rae_a,https://doi.org/10.1029/2008JA013559
6,2009-02-01,Wavelet-based ULF wave diagnosis of substorm e...,Journal of Geophysical Research: Space Physics,NaN,"Murphy, K. R., Jonathan Rae, I., Mann, I. R., ...",Murphy_a,https://doi.org/10.1029/2008JA013548
7,2009-07-01,Near-Earth initiation of a terrestrial substorm,Journal of Geophysical Research: Space Physics,NaN,"Rae, I. J., Mann, I. R., Angelopoulos, V., Mur...",Rae_b,https://doi.org/10.1029/2008JA013771
8,2009-10-01,Reply to comment by K. Liou and Y.-L. Zhang on...,Journal of Geophysical Research: Space Physics,NaN,"Murphy, K. R., Rae, I. J., Mann, I. R., Walsh,...",Murphy_b,https://doi.org/10.1029/2009JA014351
9,2010-10-01,Optical characterization of the growth and spa...,Journal of Geophysical Research: Space Physics,NaN,"Rae, I. J., Watt, C. E. J., Mann, I. R., Murph...",Rae,https://doi.org/10.1029/2010JA015376


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
        #md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
        f.write(md)

TypeError: 'float' object is not iterable

These files are in the publications directory, one directory below where we're working from.

In [31]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).